<a href="https://colab.research.google.com/github/YinYinTan/CIND820FinalProject/blob/main/Data_Transformation_%26_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pandas_profiling --upgrade

# Data Transformation

In [17]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

#Read file
df = pd.read_csv('HS.csv')

#Drop rows containing observations that are not valid for the project
df.drop(df[df['EMP_30'] != 1].index, inplace = True)
df.drop(df[df['GEN_10'] == 9].index, inplace = True)
df.drop(df[df['GEN_15'] == 9].index, inplace = True)

#Create new derived variable, which will be the target variable for this project
df['Worse_MH'] = np.where((df['GEN_10'] >= 4) & (df['GEN_15'] >= 4), 0, 1)

#Drop columns that are not needed for the project
df = df.drop(columns=['PUMFID', 'VERDATE', 'EMP_05', 'BMF_P', 'EMP_30', 'GEN_10', 'GEN_15'])

#Drop columns pertaining to derived variables that are not needed for the project
df = df.drop(columns=['ENVDVCON','PPEDVEY1','PPEDVEY2','PPEDVFV1','PPEDVFV2','PPEDVGL1','PPEDVGL2','PPEDVGN1','PPEDVGN2','PPEDVHS1','PPEDVHS2','PPEDVMA1','PPEDVMA2','PPEDVOT1','PPEDVOT2','PPEDVRE1','PPEDVRE2','PPEDVRS1','PPEDVRS2','GENDVHDI','GENDVMHI'])

#If PPE_10 = “No” = 2, then PPE_15A, PPE_15B, PPE_15C, PPE_15D, PPE_15E, PPE_15F, PPE_15G, PPE_15H, PPE_15I, and PPE_15J = “No” = 2
set_1 = ['PPE_15A','PPE_15B','PPE_15C','PPE_15D','PPE_15E','PPE_15F','PPE_15G','PPE_15H','PPE_15I','PPE_15J']
df[set_1] = np.where(df[['PPE_10']] == 2, 2, df[set_1])

#If PPE_20 = “No” = 2, then PPE_30A, PPE_30B, PPE_30C, PPE_30D, PPE_30E, PPE_30F, PPE_30G, PPE_30H,...
#PPE_35A, PPE_35B, PPE_35C, PPE_35D, PPE_35E, PPE_35F, PPE_35G, PPE_35H = "Not needed for job" = 1
set_2 = ['PPE_30A','PPE_30B','PPE_30C','PPE_30D','PPE_30E','PPE_30F','PPE_30G','PPE_30H','PPE_35A','PPE_35B','PPE_35C','PPE_35D','PPE_35E','PPE_35F','PPE_35G','PPE_35H']
df[set_2] = np.where(df[['PPE_20']] == 2, 1, df[set_2])

#If PPE_20 = “No” = 2, then PPE_25, PPE_40A, PPE_40B, PPE_40C, PPE_40D, PPE_40E, PPE_40F, PPE_40G, PPE_40H, PPE_40I,...
#PPE_45A, PPE_45B, PPE_45C, PPE_45D, PPE_45E, PPE_45F, PPE_45G, PPE_45H, PPE_45I = "No" = 2
set_3 = ['PPE_25','PPE_40A','PPE_40B','PPE_40C','PPE_40D','PPE_40E','PPE_40F','PPE_40G','PPE_40H','PPE_40I','PPE_45A','PPE_45B','PPE_45C','PPE_45D','PPE_45E','PPE_45F','PPE_45G','PPE_45H','PPE_45I']
df[set_3] = np.where(df[['PPE_20']] == 2, 2, df[set_3])

#If response to PPE access question = “Not stated” = 9 (i.e. missing values), then update to "Not needed for job" = 1
df['PPE_30A'].mask(df['PPE_30A'] == 9, 1, inplace=True)
df['PPE_30B'].mask(df['PPE_30B'] == 9, 1, inplace=True)
df['PPE_30C'].mask(df['PPE_30C'] == 9, 1, inplace=True)
df['PPE_30D'].mask(df['PPE_30D'] == 9, 1, inplace=True)
df['PPE_30E'].mask(df['PPE_30E'] == 9, 1, inplace=True)
df['PPE_30F'].mask(df['PPE_30F'] == 9, 1, inplace=True)
df['PPE_30G'].mask(df['PPE_30G'] == 9, 1, inplace=True)
df['PPE_30H'].mask(df['PPE_30H'] == 9, 1, inplace=True)
df['PPE_35A'].mask(df['PPE_35A'] == 9, 1, inplace=True)
df['PPE_35B'].mask(df['PPE_35B'] == 9, 1, inplace=True)
df['PPE_35C'].mask(df['PPE_35C'] == 9, 1, inplace=True)
df['PPE_35D'].mask(df['PPE_35D'] == 9, 1, inplace=True)
df['PPE_35E'].mask(df['PPE_35E'] == 9, 1, inplace=True)
df['PPE_35F'].mask(df['PPE_35F'] == 9, 1, inplace=True)
df['PPE_35G'].mask(df['PPE_35G'] == 9, 1, inplace=True)
df['PPE_35H'].mask(df['PPE_35H'] == 9, 1, inplace=True)

#If response to PPE restriction question = “Not stated” = 9 (i.e. missing values), then update to "No" = 2
df['PPE_40A'].mask(df['PPE_40A'] == 9, 2, inplace=True)
df['PPE_40B'].mask(df['PPE_40B'] == 9, 2, inplace=True)
df['PPE_40C'].mask(df['PPE_40C'] == 9, 2, inplace=True)
df['PPE_40D'].mask(df['PPE_40D'] == 9, 2, inplace=True)
df['PPE_40E'].mask(df['PPE_40E'] == 9, 2, inplace=True)
df['PPE_40F'].mask(df['PPE_40F'] == 9, 2, inplace=True)
df['PPE_40G'].mask(df['PPE_40G'] == 9, 2, inplace=True)
df['PPE_40H'].mask(df['PPE_40H'] == 9, 2, inplace=True)
df['PPE_40I'].mask(df['PPE_40I'] == 9, 2, inplace=True)
df['PPE_45A'].mask(df['PPE_45A'] == 9, 2, inplace=True)
df['PPE_45B'].mask(df['PPE_45B'] == 9, 2, inplace=True)
df['PPE_45C'].mask(df['PPE_45C'] == 9, 2, inplace=True)
df['PPE_45D'].mask(df['PPE_45D'] == 9, 2, inplace=True)
df['PPE_45E'].mask(df['PPE_45E'] == 9, 2, inplace=True)
df['PPE_45F'].mask(df['PPE_45F'] == 9, 2, inplace=True)
df['PPE_45G'].mask(df['PPE_45G'] == 9, 2, inplace=True)
df['PPE_45H'].mask(df['PPE_45H'] == 9, 2, inplace=True)
df['PPE_45I'].mask(df['PPE_45I'] == 9, 2, inplace=True)

#If response to PPE or IPC protocol/practice question = “Not stated” = 9 (i.e. missing values), then remove from dataset
df.drop(df[(df['PPE_05'] == 9)|(df['PPE_10'] == 9)|(df['PPE_15A'] == 9)|(df['PPE_15B'] == 9)|(df['PPE_15C'] == 9)|(df['PPE_15D'] == 9)|(df['PPE_15E'] == 9)|(df['PPE_15F'] == 9)|(df['PPE_15G'] == 9)|(df['PPE_15H'] == 9)|(df['PPE_15I'] == 9)|(df['PPE_15J'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_20'] == 9)|(df['PPE_25'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_50A'] >= 96)|(df['PPE_50B'] >= 96)|(df['PPE_50C'] >= 96)|(df['PPE_50D'] >= 96)|(df['PPE_50E'] >= 96)|(df['PPE_50F'] >= 96)].index, inplace = True)

#Handling of 14/16 remaining variables with classes of "Not stated" = 9 (i.e. missing values): Reassign to most frequent value
'''
In progress
'''


#One-hot key encoding
'''
In progress
'''

#Change all variables to categorical or else they will be considered numeric
df = df.astype('category')

#View basic summary of variables
df.describe()
#df.head()

#df.to_csv('HS - revised.csv')

,GEODVGPR,EMP_10,EMP_35,EMP_45,EMPDVGOC,EMPDVGYW,ENV_25A,ENV_25B,ENV_25C,ENV_30,ENVDVTYP,ENVDVGRW,PPE_05,PPE_10,PPE_15A,PPE_15B,PPE_15C,PPE_15D,PPE_15E,PPE_15F,PPE_15G,PPE_15H,PPE_15I,PPE_15J,PPE_20,PPE_25,PPE_30A,PPE_30B,PPE_30C,PPE_30D,PPE_30E,PPE_30F,PPE_30G,PPE_30H,PPE_35A,PPE_35B,PPE_35C,PPE_35D,PPE_35E,PPE_35F,PPE_35G,PPE_35H,PPE_40A,PPE_40B,PPE_40C,PPE_40D,PPE_40E,PPE_40F,PPE_40G,PPE_40H,PPE_40I,PPE_45A,PPE_45B,PPE_45C,PPE_45D,PPE_45E,PPE_45F,PPE_45G,PPE_45H,PPE_45I,PPE_50A,PPE_50B,PPE_50C,PPE_50D,PPE_50E,PPE_50F,GEN_05,GEN_20,AGEDVG4,GDRDVGRP,ISDVFLAG,PGDVFLA,IMMDVGST,Worse_MH
count,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319
unique,7,3,3,4,10,4,3,3,3,4,7,8,3,2,2,2,2,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,6,6,6,6,6,6,5,3,3,3,3,2
top,30,2,1,1,5,1,2,1,2,2,1,30,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,2,1,2,2,2,2,1,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,2,4,1,2,2,2,1,1
freq,7898,13268,12341,12613,6443,5661,10207,11599,15596,10482,7259,7879,14572,16534,15527,15123,13247,15136,13074,13836,10252,12874,14392,9437,16810,13718,13394,14131,7234,9984,9790,9540,13783,13166,13787,15001,7273,10769,10757,10237,14354,12555,9617,10148,14165,12502,16990,14910,15823,15405,12555,12438,12165,15059,14274,17135,15974,16541,16353,10131,7423,7229,6296,7189,4061,5335,6905,7245,5012,15246,16751,15036,15064,11929


# Exploratory Data Analysis (EDA)

In [2]:
profile = ProfileReport(df)
profile.to_file(output_file="Dataset.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/pivot.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agged[v] = maybe_downcast_to_dtype(agged[v], data[v].dtype)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]